In [ ]:
import spotipy
import spotipy.oauth2 as oauth2
import pandas as pd

credentials = oauth2.SpotifyClientCredentials(
        client_id='bc98f63798b24333bfe620e8b18f99e2',
        client_secret='77491b6e4d4d47cfa4fc70d3686d5ee4')

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

In [7]:
measure_features = ['danceability', 'energy', 'key', 'loudness', 
                    'mode', 'speechiness', 'acousticness', 'instrumentalness', 
                    'liveness', 'valence', 'tempo', 'time_signature']

genres = pd.read_csv('all_genres.csv')
genres.head()

,Unnamed: 0,GENRE,SIM_GENRES,SIM_WEIGHTS,OPP_GENRES,OPP_WEIGHTS,REL_ARTISTS,ARTIST_WEIGHTS,SPOTIFY_URL
0,0,backgroundmusic,"['reiki', 'calminginstrumental', 'yoga', 'back...","['103', '120', '101', '240', '102', '125', '11...","['mahraganat', 'divahouse', 'oldschoolukhiphop...","['160', '100', '121', '110', '135', '136', '12...","['NoemiNucci', 'JuditheAbelsen', 'FabienneWill...","['113', '117', '106', '131', '109', '115', '11...",https://open.spotify.com/user/thesoundsofspoti...
1,1,classicmoroccanpop,"['bluesrock', 'andalusianclassical', 'chaabima...","['103', '107', '109', '100', '102', '103', '10...","['shiverpop', 'gravewave', 'tassieindie', 'est...","['112', '112', '105', '102', '124', '100', '16...","['HajHusseinToulali', 'AbdelhadiBelkheyat', 'H...","['102', '103', '103', '108', '108', '102', '10...",https://open.spotify.com/user/thesoundsofspoti...
2,2,orthodoxchant,"['universitychoir', 'polishchoir', 'ukrainianc...","['102', '105', '107', '104', '106', '240', '10...","['sambass', 'funkybreaks', 'hardbass', 'hardco...","['107', '110', '158', '100', '119', '111', '10...","['LegeArtis', 'PeterMichaelides', 'CapellaGreg...","['114', '100', '160', '127', '115', '115', '10...",https://open.spotify.com/user/thesoundsofspoti...
3,3,tar,"['oud', 'persiantraditional', 'bansuri', 'tar'...","['106', '120', '101', '240', '103', '113', '10...","['happyhardcore', 'aggrotech', 'nightcore', 'b...","['109', '107', '127', '114', '115', '120', '12...","['KiyarashSaket', 'YusefForutan', 'HamidMoteba...","['100', '100', '111', '100', '100', '100', '10...",https://open.spotify.com/user/thesoundsofspoti...
4,4,reggae,"['dancehall', 'brazilianreggae', 'skarevival',...","['109', '105', '114', '101', '100', '127', '11...","['brazilianlo-firock', 'martialindustrial', 'p...","['106', '107', '155', '121', '160', '108', '14...","['TheItals', 'MykalRose', 'JohnnieClark', 'Asw...","['101', '102', '100', '105', '101', '105', '10...",https://open.spotify.com/user/thesoundsofspoti...


In [8]:
genres['PLAY_ID'] = genres['SPOTIFY_URL'].map(lambda s: s.split('playlist')[1][1:])

In [9]:
base = 'https://api.spotify.com/v1/'

In [4]:
def get_tids(play_id):
    url = base+'playlists/'+play_id+'/tracks'
    playlist_tracks = spotify._get(url = url)
    tids = []
    for track in playlist_tracks['items']:
        try:
            tids.append(track['track']['id'])
        except:
            pass
    return tids


In [5]:
def get_avg_features(df):
    rows = []
    dicts = []
    for playlist in df['PLAY_ID'].values:
        print(playlist, len(rows))
        tids = get_tids(playlist)
        url = base+'audio-features/?ids=' + ','.join(tids)
        audio = spotify._get(url = url)
        try:
            df = pd.DataFrame(audio['audio_features'])
            df = df[measure_features]
            feature_means = df.describe().loc['mean']
        except:
            audio_feature_list = [i for i in audio['audio_features'] if i]
            df = pd.DataFrame(audio_feature_list)
            df = df[measure_features]
            feature_means = df.describe().loc['mean']
            feature_means = pd.Series({k:-1 for k in measure_features})
        dicts.append(feature_means.to_dict()) 
        feature_means.name = playlist
        rows.append(feature_means)
        if len(rows)%100 == 0:
            print('Covered {} rows'.format(len(rows)))
    return rows,dicts

In [ ]:
test = genres.head(20)
rows, dicts = get_avg_features(test)
test['PLAYLIST_FEATURES'] = dicts
avg_features = pd.DataFrame(rows)
avg_features

In [ ]:
len(genres)

In [ ]:
chunk1 = genres.head(1000)
chunk2 = genres.head(2000).tail(1000)
chunk3 = genres.head(3000).tail(1000)
chunk4 = genres.tail(652)

In [ ]:
rows, dicts = get_avg_features(chunk1)

In [ ]:
rows2, dicts2 = get_avg_features(chunk2)

In [ ]:
rows3, dicts3 = get_avg_features(chunk3)

In [ ]:
rows4, dicts4 = get_avg_features(chunk4)

In [ ]:
all_dicts = dicts + dicts2 + dicts3 + dicts4

In [ ]:
all_rows = rows + rows2 + rows3 + rows4
df = pd.DataFrame(all_rows)
df

In [ ]:
df.to_csv('play_id_to_features.csv')